In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import math
import collections

# Load the trained model
model = load_model('sign_language_alphaDigi_model.h5')

# Initialize video capture and hand detector
cap = cv2.VideoCapture(0)
detector = HandDetector(
    maxHands=2,
    detectionCon=0.8,  # Higher detection confidence
    minTrackCon=0.5    # Minimum tracking confidence
)

# Constants
offset = 20
imgSize = 128
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

def preprocess_image(img):
    img = cv2.resize(img, (imgSize, imgSize))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype('float32') / 255.0
    img_final = np.expand_dims(img, axis=0)
    return img_final, img  # Return both the model input and the normalized image for display

prediction_buffer = collections.deque(maxlen=5)

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    
    if hands:
        # Get bounding box coordinates for all hands
        x_min = min(hand['bbox'][0] for hand in hands) - offset
        y_min = min(hand['bbox'][1] for hand in hands) - offset
        x_max = max(hand['bbox'][0] + hand['bbox'][2] for hand in hands) + offset
        y_max = max(hand['bbox'][1] + hand['bbox'][3] for hand in hands) + offset

        # Ensure coordinates are within image bounds
        x_min, y_min = max(0, x_min), max(0, y_min)
        x_max, y_max = min(img.shape[1], x_max), min(img.shape[0], y_max)

        imgCrop = img[y_min:y_max, x_min:x_max]
        aspectRatio = (y_max - y_min) / (x_max - x_min)

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        if aspectRatio > 1:
            k = imgSize / (y_max - y_min)
            wCal = math.ceil(k * (x_max - x_min))
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wGap + wCal] = imgResize
        else:
            k = imgSize / (x_max - x_min)
            hCal = math.ceil(k * (y_max - y_min))
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hGap + hCal, :] = imgResize
        
        # Preprocess and predict
        processed_img, normalized_img = preprocess_image(imgWhite)
        prediction = model.predict(processed_img, verbose=0)
        index = np.argmax(prediction[0])
        confidence = prediction[0][index]

        prediction_buffer.append(labels[index])
        smoothed_pred = max(set(prediction_buffer), key=prediction_buffer.count)
        

        cv2.rectangle(imgOutput, (x_min, y_min - 70), (x_min + 250, y_min - 20), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, f"{smoothed_pred} ({confidence:.2f})", (x_min, y_min - 30),cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x_min, y_min), (x_max, y_max), (255, 0, 255), 4)
        
        # Show cropped and processed images
        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)
        cv2.imshow("Preprocessed Image", normalized_img)
    
    cv2.imshow("Image", imgOutput)
    
    # Break loop on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: module 'numpy' has no attribute 'dtypes'